# Doc2Vec para RI

Este enfoque toma los subtítulos de cada vídeo y los transforma a un vector generado por Dov2Vec, posteriormente para regresar los subtítulos que más se parecen a un query comparamos los vectores por su distancia y regresamos los 5 vectores que más cerca están del vector query

In [22]:
from os import listdir
from os.path import isfile, join
#Para el corpus
import sys
sys.path.append('..')
from src.utils import *
from collections import defaultdict
import json
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
from tqdm import tqdm 

In [3]:
def get_subtittle():
    """
    Regresa un diccionario con la información de cada subtítulo 
    (id del video: lista de subtítulos)
    y otro diccionario que contiene los subtítulos, las llaves de estos 
    subtítulos son de la forma: idvideo-idsubtítulo, esto para poder regresar 
    la información con el primer diccionario
    """
    path = "../corpus/data"
    sub = defaultdict()
    info = defaultdict()
    onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]

    for file in onlyfiles:
        with open(path+"/"+file, 'rb') as f:
            chanel = json.loads(f.read().decode('utf-8', 'replace'))
            for video in chanel:
                if 'subtitles' in video: #Si hay subtítulos
                    info[video["id"]] = []
                    for i, s in enumerate(video['subtitles']):
                        name = video["id"] + "-" + str(i) #creamos llave del subtítulo
                        sub[name] = get_tokens_clean(s['text'], stop_words=True) #Limpiamos
                        info[video["id"]].append(s)
    return sub, info

In [4]:
sub, info = get_subtittle()

In [5]:
print(sub["68tp7jWrcjA-2"], info["68tp7jWrcjA"][2])

['frida', 'kahlo', 'andaba', 'regreso', 'escuela', 'ciudad', 'méxico'] {'start': '7.791', 'dur': '3.09', 'text': 'Frida Kahlo andaba de regreso de\nla escuela en la ciudad de México'}


#### Preparamos los datos para utilizar el objeto doc2vec
(Esta parte se puede omitir si ya se entrenó antes)

In [37]:
# Convertimos los tokens de cada título en el formato requerido por gensim 
tagged_data = [TaggedDocument(d, [i]) for i, d in sub.items()]
tagged_data[:5]

[TaggedDocument(words=['adónde', 'vas', 'momo'], tags=['_DZF9EqExZM-0']),
 TaggedDocument(words=['débil'], tags=['_DZF9EqExZM-1']),
 TaggedDocument(words=['crees', 'podrás', 'sobrevivir'], tags=['_DZF9EqExZM-2']),
 TaggedDocument(words=['cumples', 'reglas', 'sociedad'], tags=['_DZF9EqExZM-3']),
 TaggedDocument(words=['vencido', 'encontramos', 'guarida'], tags=['_DZF9EqExZM-4'])]

#### Entrenamos y guardamos
(Esta parte se puede omitir si ya se entrenó antes)

In [38]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size=10, window=2, min_count=1, workers=4, epochs = 100)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
model.save("models/sub_doc2vec.model")

#### Cargamos el modelo

In [7]:
## Load saved doc2vec model
model= Doc2Vec.load("models/sub_doc2vec.model")
print(len(model.dv))

540324


In [29]:
def get_top5(query, show=False):
    """
    Regresa los 5 subtítulos que más se parecen a una query

    Args:
        query (str): cadena a buscar
        show (bool): muestra en pantalla el resultado?
    """
    test_doc = get_tokens_clean(query) #Limpiamos la query
    #Obtenemos los 5 primeros
    resp = model.dv.most_similar(positive=[model.infer_vector(test_doc)],topn=5)
    info_resp = []
    for doc in resp:
        aux = doc[0].rsplit('-', 1) #Obtenemos idVideo - idSub
        aux[1] = int(aux[1])
        if show:
            print('Id: {} \nInfo: {}\n ---------'.format(aux[0],info[aux[0]][aux[1]]))
        info_resp.append(info[aux[0]][aux[1]]['text'])
    return info_resp

In [20]:
r = get_top5("Receta de pastel con almendras", True)

Id: xLI9PUjwpbM 
Info: {'start': '1.859', 'dur': '4.178', 'text': 'Una receta de pastel suave, esponjoso\ny humedo.'}
 ---------
Id: DUhvKm4VLeg 
Info: {'start': '1.668', 'dur': '3.121', 'text': 'Una receta de Pastel suave y esponjoso.'}
 ---------
Id: cHv2QxwXg9U 
Info: {'start': '642.42', 'dur': '7.1', 'text': 'el ambiente de calor en la mesa la cena'}
 ---------
Id: 8Bbgqu3O7fk 
Info: {'start': '423.9', 'dur': '5.489', 'text': 'un esqueleto que durante ni más ni menos'}
 ---------
Id: sH8PM_ehUys 
Info: {'start': '8.309', 'dur': '4.801', 'text': 'que es lo más difícil de ser una raqueta'}
 ---------


In [21]:
print(r)

['Una receta de pastel suave, esponjoso\ny humedo.', 'Una receta de Pastel suave y esponjoso.', 'el ambiente de calor en la mesa la cena', 'un esqueleto que durante ni más ni menos', 'que es lo más difícil de ser una raqueta']


#### Probamos con el conjunto test
...para posteriormente evaluarlo

In [23]:
with open("../pkl/test.pkl", "rb") as f:
    test = pickle.load(f)

In [30]:
#Consulta
test_results = []

for query in tqdm(test):
    r = get_top5(query)
    test_results.append(r)

pickle.dump(test_results, open("../pkl/resultsDoc2Vec.pkl", "wb"))

100%|██████████| 4535/4535 [02:51<00:00, 26.38it/s]


In [32]:
with open("../pkl/resultsDoc2Vec.pkl", "rb") as f:
    test_resultsxO = pickle.load(f)

for i in range(5):
    print(test[i], test_resultsxO[i])
    print("-------------")

a un lado oigan alejen a los demás viene ['a un lado oigan alejen a los demás viene', 'cruzado y deciden crear ahi unos', '¿Qué pasa?\nNo te lo guardes', '¡pero miren estas tazas!', 'Retire y colóquelos en un rack.']
-------------
saber lo que se está toda máquina ['saber lo que se está toda máquina', 'eso es hasta para orinar es lenta', 'Si no sabes mucho acerca de la termodinámica, el Pájaro Bebedor podría parecer que podría durar para siempre, sin una fuente externa de energía que lo alimente, una máquina de movimiento perpetuo, en otras palabras.', 'Ahora viene el tiempo de revisión. Puede hacer clic en cualquier de estos enlaces y volver a la parte de', 'Pero, en orden para ir de un simple tubo al clásico, arrugado icono que pensamos']
-------------
estás llorando que ya me involucre ['estás llorando que ya me involucre', 'Oye, espera. ¿Qué haces?', 'la misa no lo hagas cuando acabe la misa', 'raúl ten cuidado que cuando grabas un', 'Perdí a mi hija, perdí mi casa']
-------------
